In [12]:
from PIL import Image
import torch
import timm
import requests
import torchvision.transforms as transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()
    



Using cache found in /home/amirahsaei/.cache/torch/hub/facebookresearch_deit_main


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

In [57]:
with torch.no_grad():
    for name, param in model.named_parameters():
        param.data = torch.zeros(list(param.data.size()))

model.eval()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

In [13]:
import torch
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

val_dataset = datasets.ImageFolder(
    'IMG/val/',
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))


val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=512, shuffle=False,
    num_workers=8, pin_memory=True, sampler=None)


correct = 0
total = 0



#model.cuda()

activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def fn(_, input, output):
        #output[0][:] = 0
        output[abs(output) > 0.6] = 0
        #print('tottal count is: ', torch.numel(output), ' # zeros: ', torch.numel(output) - int(torch.count_nonzero(output)))
        return


with torch.no_grad():
    for i, (images, target) in enumerate(val_loader):

        #model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True).to('cpu')

        images = images
        target = target

        model.blocks[3].attn.qkv.register_forward_hook(fn)

        ########################################################## sparsity analysis ########################################

        h = model.blocks[3].attn.qkv.register_forward_hook(get_activation('blocks[3].attn.qkv'))

        # print('the siez is: ', activation['blocks[3].attn.qkv'].size())

        #h.remove()

        outputs = model(images)

        att = activation['blocks[3].attn.qkv']

        B, N, C = 512, 197, 768

        qkv = att.reshape(B, N, 3, 12, C // 12).permute(2, 0, 3, 1, 4)

        q, k, v = qkv.unbind(0) 

        for i in range (0, 12):
            print(f'for head {i} tottal count is: ', torch.numel(torch.chunk(q, 12, 1)[i]), ' # zeros: ', torch.numel(torch.chunk(q, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(q, 12, 1)[i])), "ratio: ", ((torch.numel(torch.chunk(q, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(q, 12, 1)[i])))/6455296)*100)
            print(f'for head {i} tottal count is: ', torch.numel(torch.chunk(k, 12, 1)[i]), ' # zeros: ', torch.numel(torch.chunk(k, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(k, 12, 1)[i])), "ratio: ", ((torch.numel(torch.chunk(k, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(k, 12, 1)[i])))/6455296)*100)
            print(f'for head {i} tottal count is: ', torch.numel(torch.chunk(v, 12, 1)[i]), ' # zeros: ', torch.numel(torch.chunk(v, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(v, 12, 1)[i])), "ratio: ", ((torch.numel(torch.chunk(v, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(v, 12, 1)[i])))/6455296)*100)
            print("******************************************************************************")
            #print(torch.chunk(q, 12, 1)[0].size())

        
        
        _, predicted = torch.max(outputs, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        break



print(f'Accuracy of the network on the {512} test images: {100 * correct // total} %')

print(total)


for head 0 tottal count is:  6455296  # zeros:  3693962 ratio:  57.22374310953363
for head 0 tottal count is:  6455296  # zeros:  3758719 ratio:  58.22690392508725
for head 0 tottal count is:  6455296  # zeros:  1098138 ratio:  17.011427516259516
******************************************************************************
for head 1 tottal count is:  6455296  # zeros:  3495215 ratio:  54.14492224678775
for head 1 tottal count is:  6455296  # zeros:  3461619 ratio:  53.62448135608344
for head 1 tottal count is:  6455296  # zeros:  1246894 ratio:  19.3158299789816
******************************************************************************
for head 2 tottal count is:  6455296  # zeros:  2548585 ratio:  39.48052885568686
for head 2 tottal count is:  6455296  # zeros:  2553055 ratio:  39.549774324833436
for head 2 tottal count is:  6455296  # zeros:  1855245 ratio:  28.73989047132773
******************************************************************************
for head 3 tottal count 

: 

In [37]:
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

In [1]:
x = torch.randn(1, 3, 224, 224)

NameError: name 'torch' is not defined

In [5]:
activation = {}
fm_global = []
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.to('cpu')
print(model.blocks[0].norm1)



def fn(_, input, output):
        #output[0][:] = 0
        output[abs(output) < 0.1] = 0
        print('tottal count is: ', torch.numel(output), ' # zeros: ', torch.numel(output) - int(torch.count_nonzero(output)))
        return

output = model(x)
print(output[0][:10])

#model.blocks[0].norm1.register_forward_hook(fn)
model.blocks[3].attn.qkv.register_forward_hook(fn)

h = model.blocks[3].attn.qkv.register_forward_hook(get_activation('blocks[3].attn.qkv'))

print('the siez is: ', activation['blocks[3].attn.qkv'].size())

h.remove()

att = activation['blocks[3].attn.qkv']

output = model(x)

print("****************************************************************")

print(att.size())

B, N, C = 1, 197, 768

qkv = att.reshape(B, N, 3, 12, C // 12).permute(2, 0, 3, 1, 4)

q, k, v = qkv.unbind(0) 

print(q.size())
for i in range (0, 12):
    print(f'for head {i} tottal count is: ', torch.numel(torch.chunk(q, 12, 1)[i]), ' # zeros: ', torch.numel(torch.chunk(q, 12, 1)[i]) - int(torch.count_nonzero(torch.chunk(q, 12, 1)[i])))
    #print(torch.chunk(q, 12, 1)[0].size())

print('tottal count is: ', torch.numel(q), ' # zeros: ', torch.numel(q) - int(torch.count_nonzero(q)))
print('tottal count is: ', torch.numel(k), ' # zeros: ', torch.numel(k) - int(torch.count_nonzero(k)))
print('tottal count is: ', torch.numel(v), ' # zeros: ', torch.numel(v) - int(torch.count_nonzero(v)))

#print(output[0][:10])

LayerNorm((768,), eps=1e-06, elementwise_affine=True)


: 

: 

In [29]:
t = torch.tensor([-10, 1, 0.7, 1.2, 10])

t[abs(t) > 5] = 0


print(t)

tensor([0.0000, 1.0000, 0.7000, 1.2000, 0.0000])
